In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/drd2/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/drd2/test_1.csv', index_col=0)

train

,smiles,value
383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True
386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True
389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True
2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True
2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False
...,...,...
5444,O=C1c2ccccc2C(=O)N1CCCCN1CCCN(C(c2ccccc2)c2ccc...,True
4391,O=C(CCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
4397,O=C(CCCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
5999,OC12C3C4CC5C6C4C1C6C(C53)N2CC1CCCCC1,False


# Hyperparameter Optimization

In [4]:
def run_gb_gridsearch(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 500],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.4, 0.7, 0.9, 1.0],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
    'max_features': [None, 'sqrt']
    }
    knn = GradientBoostingClassifier()

    grid_search = RandomizedSearchCV(knn, params, cv=pds, n_iter=30, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    knn = GradientBoostingClassifier(**best_params)
    knn.fit(train_fps, train['value'])

    test_preds = knn.predict_proba(test_fps)[:, 1]
    test_metrics = get_hi_metrics(test, test_preds)
    return test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

test_metrics = run_gb_gridsearch(train_maccs_fps, test_maccs_fps)
print(test_metrics)

Fitting 1 folds for each of 30 candidates, totalling 30 fits
[CV 1/1] END learning_rate=0.7, max_depth=4, max_features=sqrt, min_samples_leaf=3, min_samples_split=7, n_estimators=50, subsample=0.4;, score=0.650 total time=   1.4s
[CV 1/1] END learning_rate=0.7, max_depth=4, max_features=sqrt, min_samples_leaf=3, min_samples_split=7, n_estimators=50, subsample=0.9;, score=0.673 total time=   1.4s
[CV 1/1] END learning_rate=0.01, max_depth=2, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.7;, score=0.618 total time=   2.2s
[CV 1/1] END learning_rate=1.0, max_depth=3, max_features=sqrt, min_samples_leaf=3, min_samples_split=5, n_estimators=150, subsample=0.4;, score=0.647 total time=   1.4s
[CV 1/1] END learning_rate=0.3, max_depth=2, max_features=None, min_samples_leaf=3, min_samples_split=7, n_estimators=150, subsample=0.7;, score=0.635 total time=   1.9s
[CV 1/1] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_s

# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

    gb = GradientBoostingClassifier(
        n_estimators=150,
        subsample=0.9,
        min_samples_split=2,
        min_samples_leaf=5,
        max_features='sqrt',
        max_depth=3,
        learning_rate=0.7
    )
    gb.fit(train_maccs_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = train_result['value']

    test_result = test.copy()
    test_result['preds'] = gb.predict_proba(test_maccs_fps)[:, 1]

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/drd2/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/drd2/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/drd2/gb_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/drd2/gb_maccs/test_{i}.csv')
